# Epymix integration

This tutorial demonstrate how an existing openalea model (epymix) can be exported as an IPM-compliant web service

## Get Epymix factory from OpenAlea package manager

In [1]:
from openalea.core.pkgmanager import PackageManager
pm = PackageManager()
pm.init()

In [2]:
factory = pm['ipmdecisions.epymix']['epymix_seir']
node = factory.instantiate()

In [3]:
[v['name'] for v in node.input_desc], node.output_desc

(['sowing_date',
  'daily_tmin',
  'daily_tmax',
  'daily_rain',
  'delta_t',
  'rainfall_threshold',
  'scenario_ino',
  'Lx',
  'Ly',
  'Lr',
  'frac_inf',
  'inoc_init_abs',
  'ng_ext0_abs',
  'scenario_rot',
  'wheat_fraction',
  'day_length',
  'alpha_ure',
  'radius_ure',
  'alpha_asco',
  'radius_asco',
  'alpha_pycnid',
  'radius_pycnid',
  't',
  'season',
  'mu_companion',
  'beta_companion',
  'end_companion',
  'LAI_K',
  'delta_companion',
  'disease',
  'mu_wheat',
  'nu',
  'beta_wheat',
  'end_wheat',
  'ber_wheat',
  'ber_companion',
  'h_wheat',
  'h_companion',
  'lambd',
  'delta_ei',
  's0',
  'pi_inf0',
  'rho',
  'psi',
  'gamma',
  'theta',
  'sigma',
  'sigma_asco',
  'inf_begin'],
 [{'name': 'Nsp', 'interface': 'ISequence'},
  {'name': 'Pth', 'interface': 'ISequence'},
  {'name': 'Poi', 'interface': 'ISequence'},
  {'name': 'Sth', 'interface': 'ISequence'},
  {'name': 'Sus', 'interface': 'ISequence'},
  {'name': 'Lat', 'interface': 'ISequence'},
  {'name': 'If

In [4]:
factory.get_id(), factory.get_authors()

('epymix_SEIR',
 'Sebastien Levionnois, Christophe Pradal, ... (wralea authors)')

In [5]:
factory.get_node_file().replace('\\','/')

'c:/users/fourniec/develop/ipm/epymix/src/epymix/wralea/adaptor.py'

## Create IPM model

In [6]:
from openalea.dss.dss_factory import dss_factory
help(dss_factory)

Help on function dss_factory in module openalea.dss.dss_factory:

dss_factory(model_id, node, factory=None, interval=86400, weather_parameters=None, parameters=None, decision_support=None, meta=None)
    Transform an openalea node in a IPM model json descriptor and generate a fastAPI script to launch webservice
    
    Args:
        node: the node to be exported
        factory: the node factory associated to the node (optional)
        interval: the time step of the model (s)
        weather_parameters: a mapping between node input name and weather data codes, if any.
        None if none of the input is a weather data
        parameters: a list of node input name to be exposed as config parameters in IPM-Decison platform
        meta : model meta informations
    Returns:
        model: a json-like dict describing the model
        dss_service: a string containing the script to be run for launching the web service



In [7]:
parameters = ['sowing_date', 'delta_t','rainfall_threshold','scenario_ino','Lx','Ly','Lr','frac_inf','inoc_init_abs','ng_ext0_abs','scenario_rot','wheat_fraction',]
ipm_model, service = dss_factory('epymix', node, factory=factory,weather_parameters={'daily_tmin':1003,'daily_tmax':1004,'daily_rain':2001},parameters=parameters)

In [8]:
ipm_model

{'id': 'epymix',
 'name': 'Sample model',
 'version': '0.0.0',
 'purpose': 'Sample meta informations generated by openalea dss facctory',
 'description': 'Please provide here a concise description of the model',
 'type_of_decision': 'Short-term tactical',
 'type_of_output': 'Risk indication',
 'description_url': None,
 'citation': None,
 'keywords': None,
 'platform_validated': False,
 'pests': [],
 'crops': [],
 'authors': [{'name': 'Anonymous',
   'email': 'anonymous@mail',
   'organization': 'Unknown'}],
 'valid_spatial': {'countries': [], 'geoJSON': '{}'},
 'execution': {'type': 'ONTHEFLY',
  'endpoint': 'http://127.0.0.1:8000/epymix/',
  'form_method': 'post',
  'content_type': 'application/json',
  'input_schema_categories': {'hidden': ['modelId'],
   'system': [],
   'user_init': [],
   'triggered': [],
   'internal': []},
  'input_schema': {'type': 'object',
   'properties': {'configParameters': {'title': 'Configuration parameters',
     'type': 'object',
     'properties': {'t

In [9]:
print(service)

from __future__ import annotations

from typing import List

from pydantic import BaseModel


class ConfigParameters(BaseModel):
    timeZone: str
    timeStart: str
    timeEnd: str
    sowing_date: str
    delta_t: int
    rainfall_threshold: int
    scenario_ino: str
    Lx: int
    Ly: int
    Lr: int
    frac_inf: int
    inoc_init_abs: int
    ng_ext0_abs: int
    scenario_rot: str
    wheat_fraction: float


class LocationWeatherDatum(BaseModel):
    longitude: float
    latitude: float
    altitude: float
    data: List[List[float]]
    length: int
    width: int


class WeatherData(BaseModel):
    timeStart: str
    timeEnd: str
    interval: int
    weatherParameters: List[int]
    locationWeatherData: List[LocationWeatherDatum]


class PydanticModel(BaseModel):
    modelId: str
    configParameters: ConfigParameters
    weatherData: WeatherData


from fastapi import FastAPI
from openalea.dss.dss_factory import encode_input

app = FastAPI()

from openalea.core.node import Fun

## Launch the Webservice

In [10]:
from openalea.dss.dss_factory import start_service, stop_service

http, handler = start_service(service)

In [11]:
from IPython.display import IFrame

In [12]:
IFrame(http + '/docs', 800,400)

## Test the exported model

In [13]:
import agroservices.ipm.fakers as ipm_fakers
from agroservices.ipm.ipm import IPM

In [14]:
ipm = IPM()
input_data = ipm_fakers.input_data(ipm_model)
input_data

{'configParameters': {'timeZone': 'Europe/Oslo',
  'timeStart': '2023-03-01',
  'timeEnd': '2023-09-01',
  'sowing_date': '2019-09-01',
  'delta_t': 10,
  'rainfall_threshold': 3,
  'scenario_ino': 'initial_inoculum',
  'Lx': 1,
  'Ly': 1,
  'Lr': 1,
  'frac_inf': 1,
  'inoc_init_abs': 20000000,
  'ng_ext0_abs': 20000,
  'scenario_rot': 'uniform',
  'wheat_fraction': 0.5},
 'modelId': 'epymix',
 'weatherData': {'weatherParameters': [1003, 1004, 2001],
  'timeStart': '2023-03-01T00:00:00+01:00',
  'timeEnd': '2023-09-01T00:00:00+02:00',
  'interval': 86400,
  'locationWeatherData': [{'longitude': 176.11984235830894,
    'latitude': 36.97610336980519,
    'altitude': 0,
    'data': [[7.0, 2.4, 4.6],
     [5.7, 2.8, 1.5],
     [1.9, 2.3, 2.1],
     [7.9, 9.3, 2.8],
     [6.8, 0.6, 2.3],
     [7.7, 4.7, 2.5],
     [6.3, 6.1, 8.9],
     [3.1, 9.9, 5.2],
     [9.5, 4.0, 1.6],
     [7.2, 1.3, 3.9],
     [9.1, 6.4, 6.5],
     [6.0, 0.4, 2.0],
     [6.0, 7.8, 1.3],
     [9.4, 9.8, 9.2],
     [5

In [15]:
ipm.run_model(ipm_model, input_data)

[[[0.0,
   19714285.714285716,
   20000.0,
   0.0,
   18783023.389236398,
   18460322.12212516,
   18159936.073873173,
   17838756.550588522,
   20000.0,
   17220823.608062774,
   16924602.942187946,
   16606434.085639589,
   16312280.39870725,
   0.0,
   15703546.6777139,
   15388345.769556036,
   15103080.506051047,
   14798114.984307764,
   14526142.269187735,
   14231347.571264446,
   13956537.39065634,
   13681357.9076006,
   13425301.821116636,
   13158354.017203175,
   12300.11591741733,
   0.0,
   12356096.845210133,
   12097992.870661154,
   11858116.62460498,
   0.0,
   11328422.47469275,
   11065362.699001482,
   10828653.504603267,
   10567051.916508071,
   10335137.0416797,
   0.0,
   9837142.147552837,
   9581926.372655507,
   9368160.932341091,
   9106546.407372482,
   8875769.681760186,
   0.0,
   8362815.846024286,
   8098633.937522787,
   7888765.433541467,
   7627897.431445991,
   7433574.92883575,
   7181038.919789946,
   7011531.280436082,
   6764200.542178754,
   

# Stop service

In [16]:
stop_service(handler)

In [18]:
IFrame(http + '/docs', 800,400)